In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import tensorflow as tf
# from tensorflow.keras.layers import StringLookup

# Load data
train_data = pd.read_csv("train-data.tsv", sep='\t')
test_data = pd.read_csv("valid-data.tsv", sep='\t')

train_labels = train_data.iloc[:, 0]
train_features = train_data.iloc[:, 1]

test_labels = test_data.iloc[:, 0]
test_features = test_data.iloc[:, 1]

# Use StringLookup to encode labels
label_lookup = tf.keras.layers.StringLookup(vocabulary=['ham', 'spam'], num_oov_indices=0, mask_token=None)

# Encode labels
train_labels = label_lookup(train_labels)
test_labels = label_lookup(test_labels)

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

max_len = 100  # Set a maximum length for the sequences
trunc_type = 'post'  # Truncate sequences from the end
oov_token = "<OOV>"  # Out of vocabulary token

tokenizer = Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(train_features)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_features)
test_sequences = tokenizer.texts_to_sequences(test_features)

# Pad sequences to a uniform length
train_padded = pad_sequences(train_sequences, maxlen=max_len, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_len, truncating=trunc_type)

In [ ]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=16),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(36, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_padded,
          train_labels,
          epochs=100,
          validation_data=(test_padded, test_labels))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # Tokenize and pad the input text
    pred_sequence = tokenizer.texts_to_sequences([pred_text])
    pred_padded = pad_sequences(pred_sequence, maxlen=max_len, truncating=trunc_type)

    # Make predictions
    predictions = model.predict(pred_padded)[0][0]

    # Return a list containing prediction and label
    if predictions >= 0.5:
        return [predictions.astype(int), 'spam']
    else:
        return [predictions.astype(int), 'ham']

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      print(prediction[1])
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
